# 🎯 Beginner Object Detection Project
## Detect colored objects using webcam, draw shapes, and label them!

**What this project does:**
- Opens your webcam
- Detects colored objects (red, blue, green, yellow)
- Draws rectangles around detected objects
- Labels each object with its color and size
- Press 'q' to quit, 's' to save frame

In [1]:
# ======================= IMPORT LIBRARIES =======================
# Import required libraries
import cv2  # OpenCV for computer vision
import numpy as np  # Numpy for array operations

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


### 📚 Understanding HSV Color Space

**Why HSV instead of BGR?**
- HSV = Hue, Saturation, Value
- **Hue** = The actual color (0-180 in OpenCV)
- **Saturation** = How vivid the color is (0-255)
- **Value** = How bright it is (0-255)

**Benefits:**
- Makes it MUCH easier to detect specific colors
- Works in different lighting conditions
- RGB/BGR values change a lot with brightness, HSV doesn't

In [2]:
# ======================= DEFINE COLOR RANGES =======================
# These define what colors we're looking for in HSV format
# Format: [Hue, Saturation, Value]

# Red appears at both ends of the hue spectrum (0-10 and 160-180)
lower_red1 = np.array([0, 100, 100])      # Lower bound for red (0-10 degrees)
upper_red1 = np.array([10, 255, 255])     # Upper bound
lower_red2 = np.array([160, 100, 100])    # Red also appears at 160-180 degrees
upper_red2 = np.array([180, 255, 255])

# BLUE color range
lower_blue = np.array([100, 100, 100])    # Blue hue is around 100-130
upper_blue = np.array([130, 255, 255])

# GREEN color range
lower_green = np.array([40, 50, 50])      # Green hue is around 40-80
upper_green = np.array([80, 255, 255])

# YELLOW color range
lower_yellow = np.array([20, 100, 100])   # Yellow hue is around 20-40
upper_yellow = np.array([40, 255, 255])

print("✅ Color ranges defined!")

✅ Color ranges defined!


In [4]:
# ======================= DETECTION FUNCTION =======================
def detect_and_label_objects(frame, lower_color, upper_color, color_name, color_bgr):
    """
    Detects objects of a specific color and draws rectangles + labels
    
    Parameters:
    - frame: The current webcam image
    - lower_color: Lower HSV threshold for the color
    - upper_color: Upper HSV threshold for the color
    - color_name: Name to display (e.g., "RED")
    - color_bgr: Color for drawing rectangle in BGR format
    
    Returns:
    - Number of objects detected
    """
    
    # STEP 1: Convert frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # STEP 2: Create a mask (black & white image)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # STEP 3: Clean up the mask using morphological operations
    # Morphological operations remove noise and fill gaps
    kernel = np.ones((5, 5), np.uint8)  # 5x5 square kernel
    
    # Erosion: Removes small white noise/dots
    mask = cv2.erode(mask, kernel, iterations=1)
    
    # Dilation: Enlarges the remaining white regions and fills gaps
    mask = cv2.dilate(mask, kernel, iterations=2)
    
    # STEP 4: Find contours (outlines) of white regions in the mask
    # Contours = boundaries/edges of shapes in the image
    # RETR_EXTERNAL = only get outer contours (not nested ones)
    # CHAIN_APPROX_SIMPLE = compress contour to save memory
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Counter for detected objects
    object_count = 0
    
    # STEP 5: Loop through each detected contour
    for contour in contours:
        # Calculate area of the contour in pixels
        area = cv2.contourArea(contour)
        
        # Filter out small objects (noise) - only detect objects > 500 pixels
        # This prevents detecting tiny specks as objects
        if area > 500:
            object_count += 1
            
            # Get bounding rectangle coordinates
            # x, y = top-left corner coordinates
            # w, h = width and height of rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # STEP 6: Draw rectangle around the detected object
            # (x, y) = top-left corner, (x+w, y+h) = bottom-right corner
            # color_bgr = color of the rectangle, 3 = thickness
            cv2.rectangle(frame, (x, y), (x + w, y + h), color_bgr, 3)
            
            # STEP 7: Create label text with color name and area
            label = f"{color_name} Object"
            size_label = f"Area: {int(area)}"
            
            # STEP 8: Draw filled rectangle background for text
            # This makes text readable on any background
            # -1 thickness = filled rectangle
            cv2.rectangle(frame, (x, y - 40), (x + 200, y), color_bgr, -1)
            
            # STEP 9: Put text labels on the image
            # White text (255, 255, 255) on colored background
            cv2.putText(frame, label, (x + 5, y - 25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, size_label, (x + 5, y - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # STEP 10: Draw a small circle at the center of the object
            # This marks the exact center point
            center_x = x + w // 2  # // means integer division
            center_y = y + h // 2
            cv2.circle(frame, (center_x, center_y), 5, (0, 255, 255), -1)
    
    return object_count

print("✅ Detection function created!")

✅ Detection function created!


In [ ]:
# ======================= MAIN WEBCAM PROGRAM =======================

# STEP 1: Initialize webcam
# 0 = first webcam, 1 = second webcam (if you have multiple)
cap = cv2.VideoCapture(0)

# Check if webcam opened successfully
if not cap.isOpened():
    print("❌ Error: Could not open webcam!")
    print("💡 Tip: Make sure no other app is using the camera")
else:
    print("✅ Webcam opened successfully!")
    print("\n🎮 CONTROLS:")
    print("   - Press 'q' to QUIT")
    print("   - Press 's' to SAVE current frame")
    print("   (Red, Blue, Green, or Yellow objects work best)\n")

# Frame counter for saving images
frame_count = 0

# STEP 2: Main loop - runs until you press 'q'
while True:
    # STEP 3: Read a frame from the webcam
    # ret = True if frame captured successfully, False otherwise
    # frame = the actual image/frame from webcam
    ret, frame = cap.read()
    
    # Check if frame was read successfully
    if not ret:
        print("❌ Error: Failed to grab frame")
        break
    
    # STEP 4: Create a copy for display (so we don't modify original)
    display_frame = frame.copy()
    
    # STEP 5: Detect RED objects (special handling because red wraps in HSV)
    red_count = 0
    
    # Convert to HSV for red detection
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Create two masks for red (because red appears at 0-10 AND 160-180 in HSV)
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    
    # Combine both red masks using bitwise OR
    # This catches all red pixels regardless of where they fall in HSV
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    
    # Apply morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask_red = cv2.erode(mask_red, kernel, iterations=1)  # Remove noise
    mask_red = cv2.dilate(mask_red, kernel, iterations=2)  # Fill gaps
    
    # Find contours in the red mask
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process each red contour
    for contour in contours_red:
        area = cv2.contourArea(contour)
        
        # Only process objects larger than 500 pixels
        if area > 500:
            red_count += 1
            
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(contour)
            
            # Draw rectangle around red object
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
            
            # Draw background for text
            cv2.rectangle(display_frame, (x, y - 40), (x + 200, y), (0, 0, 255), -1)
            
            # Add labels
            cv2.putText(display_frame, "RED Object", (x + 5, y - 25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(display_frame, f"Area: {int(area)}", 
                       (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Draw center point
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.circle(display_frame, (center_x, center_y), 5, (0, 255, 255), -1)
    
    # STEP 6: Detect other colors using our function
    blue_count = detect_and_label_objects(display_frame, lower_blue, upper_blue, 
                                          "BLUE", (255, 0, 0))
    green_count = detect_and_label_objects(display_frame, lower_green, upper_green, 
                                           "GREEN", (0, 255, 0))
    yellow_count = detect_and_label_objects(display_frame, lower_yellow, upper_yellow, 
                                            "YELLOW", (0, 255, 255))
    
    # STEP 7: Create semi-transparent info panel at the top
    overlay = display_frame.copy()
    info_panel_height = 100
    
    # Draw dark rectangle at top
    cv2.rectangle(overlay, (0, 0), (display_frame.shape[1], info_panel_height), 
                 (50, 50, 50), -1)
    
    # Blend overlay with original (0.7 = 70% overlay, 0.3 = 30% original)
    display_frame = cv2.addWeighted(overlay, 0.7, display_frame, 0.3, 0)
    
    # STEP 8: Display object counts on info panel
    total_objects = red_count + blue_count + green_count + yellow_count
    
    # Main title showing total count
    cv2.putText(display_frame, f"Total Objects Detected: {total_objects}", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Individual color counts
    cv2.putText(display_frame, 
               f"Red: {red_count} | Blue: {blue_count} | Green: {green_count} | Yellow: {yellow_count}", 
               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    # Instructions
    cv2.putText(display_frame, "Press 'q' to quit | Press 's' to save frame", 
               (10, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    # STEP 9: Show the frame in a window
    cv2.imshow('Object Detection - Color Tracking', display_frame)
    
    # STEP 10: Check for key presses
    # waitKey(1) waits 1 millisecond for a key press
    # & 0xFF extracts the last 8 bits (handles compatibility issues)
    key = cv2.waitKey(1) & 0xFF
    
    # Press 'q' to quit the program
    if key == ord('q'):
        print("\n👋 Exiting program...")
        break
    
    # Press 's' to save current frame
    elif key == ord('s'):
        filename = f'detected_objects_{frame_count}.jpg'
        cv2.imwrite(filename, display_frame)
        print(f"📸 Saved: {filename}")
        frame_count += 1

# STEP 11: Cleanup - release webcam and close windows
# This is VERY important to free up resources!
cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close all OpenCV windows

print("\n✅ Program completed successfully!")
print(f"📊 Total frames saved: {frame_count}")

✅ Webcam opened successfully!

🎮 CONTROLS:
   - Press 'q' to QUIT
   - Press 's' to SAVE current frame
   (Red, Blue, Green, or Yellow objects work best)

📸 Saved: detected_objects_0.jpg
📸 Saved: detected_objects_1.jpg
📸 Saved: detected_objects_2.jpg
📸 Saved: detected_objects_3.jpg
📸 Saved: detected_objects_4.jpg
